## What is this?

This Jupyter Notebook contains Python code for building a LSTM Recurrent Neural Network to train a simple Part-of-Speech Tagger.

This code is supplied without license, warranty or support. Feel free to do with it what you will.


## Data Import

In [1]:
import nltk
nltk.download('brown')
from nltk.corpus import brown
brown.words()

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]

In [21]:
def posmapper(postag):
  s = postag[0] 
  if s in ['N','V','A']:
    return s
  elif s == 'J':
    return 'R'
  else:
    return 'O'

training_data = []
#    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
#    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])

word_to_ix = {}
a = 10

for sent in brown.tagged_sents():
  words = []
  poss = []
  for word, pos in sent:
    words.append(word)
    poss.append(posmapper(pos))
    if word not in word_to_ix:
      word_to_ix[word] = len(word_to_ix)
  training_data.append((words,poss))
  a-=1
  if a < 1:
    break
#print(word_to_ix)
#print(training_data[0])

tag_to_ix = {"N": 0, "A": 1, "V": 2, "R": 3, "O": 4}

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long).cuda()


# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 100
HIDDEN_DIM = 100

######################################################################
# Create the model:


class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        #tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_space

######################################################################
# Train the model:


model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix)).cuda()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        print("loss={:6.5f}".format(loss))


# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)


tensor([[-3.9867e-02, -1.0894e-01, -2.5587e-02, -5.6610e-02, -1.0187e-01],
        [ 9.1220e-03, -6.5780e-02, -1.1587e-02,  1.2223e-01, -1.7225e-01],
        [-4.6596e-02, -2.6793e-01, -9.5600e-02,  1.1991e-01, -2.1613e-01],
        [-5.8207e-02, -2.3572e-01, -1.0528e-01, -2.9592e-02, -8.5423e-02],
        [ 6.6457e-02, -1.6575e-01, -3.5132e-02,  1.1918e-01, -7.9012e-02],
        [ 1.3642e-01, -1.2386e-01, -8.2653e-02,  7.7077e-02, -1.2079e-01],
        [-8.9228e-03, -1.9050e-01, -7.3165e-02,  1.8912e-02,  1.0631e-02],
        [-1.2434e-01, -1.2677e-01, -9.3699e-02,  2.5065e-02, -9.3399e-02],
        [-5.5940e-03, -1.6424e-01, -1.8196e-02,  1.0833e-01, -1.1133e-01],
        [ 1.2653e-01, -9.2730e-02, -1.1572e-01, -5.3522e-02,  1.7800e-02],
        [ 1.2416e-01,  2.2926e-04, -1.1208e-01,  8.1357e-03, -7.9195e-02],
        [ 6.6484e-02, -7.0198e-02,  3.6054e-02, -2.3761e-02, -1.4574e-01],
        [ 4.4743e-02, -1.2422e-01, -5.1992e-02,  1.9024e-02, -1.3004e-01],
        [ 5.6815e-02, -8.